In [27]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow
import optuna
import optuna.integration.lightgbm as lgb
import math

In [28]:
def objective(trial):
    train = pd.read_csv("data_transformed.csv") #reading train data
    test = pd.read_csv("data_transformed.csv") #reading test data
    train_x= train.iloc[:1000,:-1]
    test_x= test.iloc[:1000,:-1]
    train_y = train.iloc[:1000,:]['Class'] 
    test_y = test.iloc[:1000,:]['Class']
    sc = StandardScaler() #scaling data
    scaled_X_train = sc.fit_transform(train_x)
    scaled_X_test = sc.transform(test_x)
    dtrain = lgb.Dataset(scaled_X_train, label = train_y, free_raw_data = False)
    dtest  = lgb.Dataset(scaled_X_test, label = test_y, free_raw_data = False)
    
    param = {
        'objective': 'poisson',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'force_row_wise': True,
        'max_depth': -1,
        
        'max_bin': trial.suggest_int('max_bin', 1, 512),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),

        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        
        'sub_feature': trial.suggest_uniform('sub_feature', 0.0, 1.0),
        'sub_row': trial.suggest_uniform('sub_row', 0.0, 1.0)
    }
    
    # Add a callback for pruning
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse')
    
    gbm = lgb.train(
        param, 
        dtrain, 
        verbose_eval = 20,
        valid_sets = [dtest], 
        callbacks = [pruning_callback]
        )
    
    preds = gbm.predict(scaled_X_test)
    accuracy = (sklearn.metrics.mean_squared_error(test_y, preds))

    return accuracy



    

In [29]:
    study = optuna.create_study(direction = 'minimize', pruner = optuna.pruners.MedianPruner(n_warmup_steps = 10))
    study.optimize(objective, n_trials = 200)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-03-03 11:45:38,491] A new study created in memory with name: no-name-e218c8da-f238-49e1-a0ef-1d3e62047557
[I 2021-03-03 11:45:45,970] A new study created in memory with name: no-name-94a2be1e-fa20-4505-bff7-14c2acd09833
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[20]	valid_0's rmse: 0.0348245
[40]	valid_0's rmse: 0.0334153
[60]	valid_0's rmse: 0.0296891
[80]	valid_0's rmse: 0.0273318
[100]	valid_0's rmse: 0.0266166
[120]	valid_0's rmse: 0.025639
[140]	valid_0's rmse: 0.0281918
[160]	valid_0's rmse: 0.0249207
[180]	valid_0's rmse: 0.0211936
[200]	valid_0's rmse: 0.0185037
[220]	valid_0's rmse: 0.0175482
[240]	valid_0's rmse: 0.017178
[260]	valid_0's rmse: 0.0168812
[280]	valid_0's rmse: 0.0162145
[300]	valid_0's rmse: 0.0164224
[320]	valid_0's rmse: 0.0164224
[340]	valid_0's rmse: 0.016852
[360]	valid_0's rmse: 0.0175245
[380]	valid_0's rmse: 0.0162951
[400]	valid_0's rmse: 0.0157577
[420]	valid_0's rmse: 0.0145118
[440]	valid_0's rmse: 0.0135246
[460]	valid_0's rmse: 0.0137461
[480]	valid_0's rmse: 0.0127607
[500]	valid_0's rmse: 0.0118073
[520]	valid_0's rmse: 0.0107308
[540]	valid_0's rmse: 0.0102319
[560]	valid_0's rmse: 0.00958038
[580]	valid_0's rmse: 0.00942136
[600]	valid_0's rmse: 0.00881834
[620]	valid_0's rmse: 0.00904981
[640]	valid

feature_fraction, val_score: 0.009813:  14%|######4                                      | 1/7 [00:02<00:15,  2.51s/it]

[20]	valid_0's rmse: 0.0493981
[40]	valid_0's rmse: 0.0447092
[60]	valid_0's rmse: 0.0429852
[80]	valid_0's rmse: 0.0386506
[100]	valid_0's rmse: 0.043304
[120]	valid_0's rmse: 0.0381984
[140]	valid_0's rmse: 0.0365484
[160]	valid_0's rmse: 0.031036
[180]	valid_0's rmse: 0.0248309
[200]	valid_0's rmse: 0.0228513
[220]	valid_0's rmse: 0.0200331
[240]	valid_0's rmse: 0.0181708
[260]	valid_0's rmse: 0.0175055
[280]	valid_0's rmse: 0.0191921
[300]	valid_0's rmse: 0.0205068
[320]	valid_0's rmse: 0.0205786
[340]	valid_0's rmse: 0.0184437
[360]	valid_0's rmse: 0.0172567
[380]	valid_0's rmse: 0.0170109
[400]	valid_0's rmse: 0.0166293
[420]	valid_0's rmse: 0.0152175
[440]	valid_0's rmse: 0.0152881
[460]	valid_0's rmse: 0.0161077
[480]	valid_0's rmse: 0.0152723
[500]	valid_0's rmse: 0.0142544
[520]	valid_0's rmse: 0.01287
[540]	valid_0's rmse: 0.0131813
[560]	valid_0's rmse: 0.0121581
[580]	valid_0's rmse: 0.0120942
[600]	valid_0's rmse: 0.0109449
[620]	valid_0's rmse: 0.0106831
[640]	valid_0's 

feature_fraction, val_score: 0.009813:  29%|############8                                | 2/7 [00:04<00:12,  2.48s/it]

...

min_data_in_leaf, val_score: 0.000867: 100%|#############################################| 5/5 [00:11<00:00,  2.35s/it][I 2021-03-03 11:57:21,486] Trial 64 finished with value: 0.0008673065709084131 and parameters: {'min_child_samples': 10}. Best is trial 60 with value: 0.0008673065709084131.

[I 2021-03-03 11:57:21,577] Trial 4 finished with value: 7.522206879409101e-07 and parameters: {'max_bin': 368, 'num_leaves': 7, 'lambda_l1': 8.682255541849265e-07, 'lambda_l2': 1.6655262702135607e-06, 'feature_fraction': 0.5936761713309231, 'bagging_fraction': 0.5265642184872541, 'bagging_freq': 3, 'min_data_in_leaf': 46, 'min_child_samples': 77, 'sub_feature': 0.21276965204909215, 'sub_row': 0.4059055980958771}. Best is trial 0 with value: 3.274817848524986e-07.
[I 2021-03-03 11:57:28,543] A new study created in memory with name: no-name-6efe934b-2590-4541-a744-372b8a56f7bc
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=98, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=98


[I 2021-03-03 11:57:28,617] Trial 0 pruned. Trial was pruned at iteration 10.
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s][I 2021-03-03 11:57:28,864] Trial 6 pruned. Trial was pruned at iteration 0.

num_leaves, val_score: inf:   0%|                                                               | 0/20 [00:00<?, ?it/s][I 2021-03-03 11:57:29,599] Trial 26 pruned. Trial was pruned at iteration 0.

bagging, val_score: inf:   0%|                                                                  | 0/10 [00:00<?, ?it/s][I 2021-03-03 11:57:29,962] Trial 36 pruned. Trial was pruned at iteration 0.

feature_fraction_stage2, val_score: inf:   0%|                                                   | 0/3 [00:00<?, ?it/s][I 2021-03-03 11:57:30,120] Trial 39 pruned. Trial was pruned at iteration 0.



regularization_factors, val_score: inf:   0%|                                                   | 0/20 [00:00<?, ?it/s][I 2021-03-03 11:57:30,758] Trial 59 pruned. Trial was pruned at iteration 0.

min_data_in_leaf, val_score: inf:   0%|                                                          | 0/5 [00:00<?, ?it/s][I 2021-03-03 11:57:31,107] Trial 64 pruned. Trial was pruned at iteration 0.

[W 2021-03-03 11:57:31,131] Trial 5 failed because of the following error: ValueError('The best booster cannot be found. It may be found in the other processes due to resuming or distributed computing. Please set the `model_dir` argument of `LightGBMTuner.__init__` and make sure that boosters are shared with all processes.')
Traceback (most recent call last):
  File "C:\Users\Mori\AppData\Roaming\Python\Python37\site-packages\optuna\_optimize.py", line 211, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-28-499dcdad5051>", line 47, in objective
    callbacks = [pruning_call

ValueError: The best booster cannot be found. It may be found in the other processes due to resuming or distributed computing. Please set the `model_dir` argument of `LightGBMTuner.__init__` and make sure that boosters are shared with all processes.